In [20]:
from glob import glob
from tqdm import tqdm
import pandas as pd
import json
import os

In [2]:
files = glob('data/*sft*.parquet')
files

['data/train_sft-00002-of-00003-ee46ed25cfae92c6.parquet',
 'data/train_sft-00001-of-00003-0a1804bcb6ae68c6.parquet',
 'data/test_sft-00000-of-00001-f7dfac4afe5b93f4.parquet',
 'data/train_sft-00000-of-00003-a3ecf92756993583.parquet']

In [9]:
mapping = {}
for f in glob('texts.jsonl*.splitted.requested'):
    with open(f) as fopen:
        for l in fopen:
            try:
                l = json.loads(l)

                if 'Source text\nclear\nLook up details' in l['r']['result']:
                    continue
                if 'clear\nLook up details' in l['r']['result']:
                    continue
                if l['r']['result'].startswith('Source text\n'):
                    continue

                mapping[l['src']] = l['r']['result']
            except:
                pass
len(mapping)

1443861

In [24]:
for f in files:
    df = pd.read_parquet(f)
    f = os.path.split(f)[1].replace('.parquet', '.jsonl')
    with open(f, 'w') as fopen:
        for i in tqdm(range(len(df))):
            row = df['messages'].iloc[i].tolist()
            for k in range(len(row)):
                row[k]['content_ms'] = mapping.get(row[k]['content'])
            
            fopen.write(f'{json.dumps(row)}\n')

100%|██████████████████████████████████| 69289/69289 [00:02<00:00, 27895.34it/s]
